In [1]:
import pandas as pd
import numpy as np

import datetime 

import os
import tempfile
from scipy import stats
import tensorflow as tf

from google.cloud import storage

from sklearn import metrics

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

from tensorflow.keras import Model, initializers, optimizers, regularizers
from tensorflow.keras.layers import Input, Dense, Conv1D, Dropout, LSTM, TimeDistributed, Activation,Softmax
from tensorflow.keras.callbacks import ModelCheckpoint

os.chdir('/home/jupyter/physiolyx/src/')





In [3]:
data = {'bucket':'physio-bucket', 'name':'moniLabData 20-03-2020.txt'}

In [4]:
colslab = ['index','scene_index','time','ms_lastline','head_posx',"head_posy","head_posz","head_rotx","head_roty","head_rotz",
        "right_posx","right_posy","right_posz","right_rotx","right_roty","right_rotz",
        "left_posx","left_posy","left_posz","left_rotx","left_roty","left_rotz",'timedel','action']

blobDownloader(data['bucket'],data['name'],'file.txt')
table = tableReader('file.txt',cols=colslab)


Blob moniLabData 20-03-2020.txt downloaded to file.txt.


In [5]:
table = tableProcessor(table, data, serve= False)

In [6]:
table= table.loc[:,['action',"head_rotx","head_roty","head_rotz"]]

table = table[table.action != 'STILL']

table = table[0:11130]

In [7]:
table.groupby('action').size()

action
3DLEX     145
3DLFL     503
3DREX     361
3DRFL     359
EXTE     2230
FLEX     1799
LBEN     1010
LROT     1945
RBEN      864
RROT     1914
dtype: int64

## Control room

In [8]:
colspre = ["head_rotx","head_roty","head_rotz"]

timeSteps = 72
n_features = len(colspre)
n_classes = table.action.unique().size

method = 'last'

In [10]:
X = featureProcessor(table, cols=colspre, serve=False)
y, true = labelProcessor(table, timeSteps=timeSteps, step=14, method = method)

Missing features, filling with 0's
Preprocessing completed
Missing labels, filling with 0's


In [220]:
for x in range(5):
    print(x)


0
1
2
3
4


In [18]:
def modelFitter(X,y, model, epochs=1, batch_size=12):
    '''removed .hdf5 from filename'''
    
    # define loss and optimizer
    adam = optimizers.Adam(learning_rate=0.001) #(ordo learning_rate=0.01, decay=0.9)

    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['categorical_accuracy']) #or categorical_accuracy? #sparse categorical crossentropy if labels not one-hot encoded
    model.save_weights('model.h5')
    
    model.load_weights('model.h5')
    
    callbacks = ModelCheckpoint('model_ep{epoch:02d}_val{val_categorical_accuracy:.2f}', monitor='val_categorical_accuracy', verbose=0,
                                save_best_only=True, save_weights_only=True, mode='max')
    history = model.fit(X,y, validation_split = 0.2, batch_size = 12,
                epochs = epochs, callbacks = [callbacks])
    return history

In [19]:
model = SimpleLSTM(timeSteps=72, n_features=3, n_classes=10)
modelFitter(X,y,model)

Train on 632 samples, validate on 158 samples
632/632 [==============================] - 14s 22ms/sample - loss: 2.9483 - categorical_accuracy: 0.1946 - val_loss: 2.7084 - val_categorical_accuracy: 0.1329


In [22]:
os.chdir('/home/jupyter/physiolyx/src/old/')

In [214]:
def actionPredictor(bucket, df):
    global model

    # Model load which only happens during cold starts
    if model is None:
        download_blob(bucket, 'model_ep04_val0.25.index', '/tmp/model_ep04_val0.25.index')
        download_blob(bucket, 'model_ep04_val0.25.data-00000-of-00001', '/tmp/model_ep04_val0.25.data-00000-of-00001')
        model = model
        model.load_weights('/tmp/model_ep04_val0.25')
    
    class_names=['RROT', 'EXTE', 'LROT', 'FLEX', 'RBEN', 'LBEN', '3DLEX', '3DREX',
       '3DLFL', '3DRFL']
    
    actionPred = []
    
    for r in range(df.shape[0]):  
        pred = model.call(df[r:r+1])
        actionPred.append(class_names[np.argmax(pred)])

        
    return actionPred

In [82]:
weights=['model_ep04_val0.25.index','model_ep04_val0.25.data-00000-of-00001']

[upload_blob(data['bucket'],w,w) for w in weights]



NameError: name 'upload_blob' is not defined

In [170]:
def featureProcess(df, cols, timeSteps=72, step=14, serve=True):
    '''preprocessing. WORKS ONE MARKER AT THE TIME. WATCH OUT FOR OUTPUTNAMES'''

    if cols is None:
        cols = ["head_rotx","head_roty","head_rotz"]
        
    features= len(cols)
    segments = []
    coldict = {}

    #fill 0's
    if [colBlank(df, col=c) for c in cols]:
        print('Missing features, filling with 0\'s')
        df = df.fillna(0)
        
    #normalise data otherwise loss= nan
    for col in cols:
        df[col] = (df[col] - df[col].min())/(df[col].max()-df[col].min())

    #we reshape into 3d arrays of length equal to timesteps. final df is= (N*timesteps*features)
    if serve:
        
        for i in range(0, len(df)-timeSteps, timeSteps):
            
            for col in cols:

                coldict[str(col[5:])] = df[col].values[i: i + timeSteps]
                segments.append(coldict[str(col[5:])])

    else:     
        
        for i in range(0, len(df) - timeSteps, step):
            
            for col in cols:

                coldict[str(col[5:])] = df[col].values[i: i + timeSteps]
                segments.append(coldict[str(col[5:])])
   
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, timeSteps, features)

    print('Preprocessing completed')

    return reshaped_segments

In [215]:
test = table

test.shape

(11130, 5)

In [216]:
Xtest = featureProcess(test, cols=None, serve=True)
Xtest.shape

Missing features, filling with 0's
Preprocessing completed


(154, 72, 3)

In [218]:
df = test

In [213]:
table['output'].unique()

array(['3DREX', 'LBEN', 'RBEN'], dtype=object)

In [212]:
table.loc[table.action==table.output,'action'].unique()

array(['LBEN', '3DREX'], dtype=object)

In [94]:
#for every element of the list, print it 72 times and create a Series. then stack it to the df

array([[0.56367064, 0.562295  , 0.5610287 ],
       [0.5598535 , 0.55879676, 0.5578766 ],
       [0.55710226, 0.5564372 , 0.5554898 ],
       [0.5545332 , 0.5534582 , 0.5527385 ],
       [0.55220103, 0.55180925, 0.55189127],
       [0.5521737 , 0.55259275, 0.55297536],
       [0.5535129 , 0.5538408 , 0.55382264],
       [0.5534491 , 0.55315757, 0.5528843 ],
       [0.55233765, 0.55205524, 0.55153596],
       [0.5510987 , 0.55069786, 0.5502241 ],
       [0.5498961 , 0.54955906, 0.54927665],
       [0.5491309 , 0.5489487 , 0.5487665 ],
       [0.5485296 , 0.54804677, 0.54773706],
       [0.54758215, 0.54766417, 0.54775524],
       [0.5479557 , 0.5480559 , 0.5481288 ],
       [0.5484294 , 0.5487665 , 0.54907626],
       [0.5494953 , 0.54987794, 0.5501786 ],
       [0.5503881 , 0.5503517 , 0.5502514 ],
       [0.5501057 , 0.55019677, 0.55029696],
       [0.55043364, 0.55054295, 0.5506796 ],
       [0.5507707 , 0.55087095, 0.55096203],
       [0.55115336, 0.5512627 , 0.5514904 ],
       [0.

In [136]:
cols=['head_rotx','head_roty','head_rotz']
for i in range(0, len(df) - timeSteps, step):
    for col in cols:

        coldict[str(col[5:])] = table[col].values[i: i + timeSteps]

        segments.append(coldict[str(col[5:])])

72